In [1]:
from mqt.bench import CompilerSettings, QiskitSettings, TKETSettings, get_benchmark
from qiskit import *
import numpy as np

In [2]:
#def create_circuit(num_qubits: int) -> QuantumCircuit:
#    ae = AmplitudeEstimation(
#        num_eval_qubits=num_qubits - 1,  # -1 because of the to be estimated qubit
#    )
#    problem = get_estimation_problem()
#
#    qc = ae.construct_circuit(problem)
#    qc.name = "ae"
#    qc.measure_all()
#    return qc

In [3]:
n=3 #number of qubit

In [4]:
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
qc = get_benchmark(benchmark_name="ae",
                   level="nativegates",
                   circuit_size=n,
                   compiler="qiskit",
                   compiler_settings=compiler_settings,
                   provider_name="ionq",)
print("---------------ae---------------")
print(qc.draw())
print("=================================================================================================")

---------------ae---------------
global phase: 7π/8
          ┌───────┐   ┌───────────┐  ┌──────────┐             ┌───────────┐»
eval_0: ──┤ Rx(π) ├───┤0          ├──┤ Rx(-π/2) ├─────────────┤0          ├»
          ├───────┤   │           │  └──────────┘             │           │»
eval_1: ──┤ Rx(π) ├───┤  Rxx(π/2) ├───────────────────────────┤  Rxx(π/2) ├»
        ┌─┴───────┴──┐│           │┌─────────────┐┌──────────┐│           │»
     q: ┤ Ry(0.9273) ├┤1          ├┤ Rz(-0.9273) ├┤ Rx(-π/2) ├┤1          ├»
        └────────────┘└───────────┘└─────────────┘└──────────┘└───────────┘»
meas: 3/═══════════════════════════════════════════════════════════════════»
                                                                           »
«         ┌─────────┐                                                      »
«eval_0: ─┤ Rx(π/2) ├──────────────────────────────────────────────────────»
«         └─────────┘              ┌───────────┐  ┌──────────┐             »
«eval_1: ───────────────

In [5]:
from qiskit_aer import AerSimulator
from qiskit import *
from qiskit.providers.fake_provider import *
import numpy as np 
import os
from qiskit.primitives import StatevectorSampler
sampler = StatevectorSampler()
job = sampler.run([qc], shots=1000)
result = job.result()
counts = result[0].data["meas"].get_counts()
print(f" > Counts: {result[0].data["meas"].get_counts()}")

 > Counts: {'010': 203, '100': 270, '110': 80, '001': 60, '111': 96, '011': 260, '000': 26, '101': 5}


In AE, the bitstring x corresponds to the estimate:
θx=x2m
θx​=2mx​

and the amplitude estimate is:
a^=sin⁡2(πθx)
a^=sin2(πθx​)

In [6]:
def estimate_amplitude_from_counts(counts, n):
    # Get the most probable measurement outcome
    most_probable_bitstring = max(counts, key=counts.get)
    
    # Convert bitstring to integer
    x = int(most_probable_bitstring, 2)

    # Compute theta
    theta = x / (2 ** n)

    # Estimate amplitude
    a_estimated = np.sin(np.pi * theta) ** 2

    return a_estimated

In [7]:
def a_to_bitstring(a, n):
    m = n
    num1 = round(np.arcsin(np.sqrt(a)) / np.pi * 2**m)
    num2 = round((np.pi - np.arcsin(np.sqrt(a))) / np.pi * 2**m)
    if num1 != num2 and num2 < 2**m and num1 < 2**m:
        counts = {format(num1, "0"+str(m)+"b"): 0.5, format(num2, "0"+str(m)+"b"): 0.5}
    else:
        counts = {format(num1, "0"+str(m)+"b"): 1}
    return counts

def estimate_s_int_from_counts(counts):
    # Find the bitstring with highest count (most probable result)
    most_common_bitstring = max(counts, key=counts.get)
    
    # Convert bitstring to integer
    s_int = int(most_common_bitstring, 2)
    return s_int



def a_from_s_int(s_int, num_counting_qubits):
    theta = s_int * np.pi / (2 ** num_counting_qubits)
    a = np.sin(theta) ** 2
    return a



# Example usage:
a = 0.2  # known amplitude

correct_dist = a_to_bitstring(a, n)
print(correct_dist)


{'001': 0.5, '111': 0.5}


In [8]:
from fidelity import *
s_int = estimate_s_int_from_counts(counts)
estimated_a = a_from_s_int(s_int, n)
a = a_from_s_int(s_int, n)
correct_dist = a_to_bitstring(a, n)
hellinger_fidelity_with_expected(counts,correct_dist )

0.2700000000000002

hellinger_fidelity_with_expected >>  called in Polarization_fidelity >> 